 I learnt about ELI5 (https://eli5.readthedocs.io/en/latest/index.html) from Konstantin Lopuhin's kernel (https://www.kaggle.com/lopuhin/eli5-for-mercari), as well as SRK's kernel on the Quora Insincere Questions Classification Competition (https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc). 
 
 I'm learning how to use this, and the ELI5 tutorials have a simple example for the Titanic competition (https://eli5.readthedocs.io/en/latest/tutorials/xgboost-titanic.html), so i figured I'd make that into a kernel for ease of reference, because I foresee myself using this library a lot in the future. 

In [ ]:
import csv
import numpy as np
import pandas as pd

from datetime import datetime as dt

from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_log_error

from eli5 import show_weights, show_prediction

In [ ]:
with open('../input/train.csv', 'rt') as f:
    data_train = list(csv.DictReader(f))
data_train[:1]

In [ ]:
with open('../input/test.csv', 'rt') as f:
    data_test = list(csv.DictReader(f))
data_test[:1]

In [ ]:
_all_xs = [{k: v for k, v in row.items() if k != 'Survived'} for row in data_train]
_all_ys = np.array([int(row['Survived']) for row in data_train])

all_xs, all_ys = shuffle(_all_xs, _all_ys, random_state=0)
train_xs, valid_xs, train_ys, valid_ys = train_test_split(
    all_xs, all_ys, test_size=0.25, random_state=0)

print('{} items total, {:.1%} true'.format(len(all_xs), np.mean(all_ys)))

### Just using some basic feature engineering, same as the ELI5 tutorial. For a more detailed exploration, cf. my old Titanic kernels (https://www.kaggle.com/delayedkarma/basic-eda-feature-engineering-and-modeling & https://www.kaggle.com/delayedkarma/grid-search-and-rf-lb-0-80861-top-10)

In [ ]:
for x in all_xs:
    if x['Age']:
        x['Age'] = float(x['Age'])
    else:
        x.pop('Age')
    x['Fare'] = float(x['Fare'])
    x['SibSp'] = int(x['SibSp'])
    x['Parch'] = int(x['Parch'])

In [ ]:
### Load the test set, do the basic pre-processing steps same as above, and make predictions on it.

all_xs_test = [{k: v for k, v in row.items()} for row in data_test]

for x in all_xs_test:
    if x['Age']:
        x['Age'] = float(x['Age'])
    else:
        x.pop('Age')
    if x['Fare']:
        x['Fare'] = float(x['Fare'])
    else:
        x.pop('Fare')
    x['SibSp'] = int(x['SibSp'])
    x['Parch'] = int(x['Parch'])

### The CSCTransformer is used to convert the data to a CSC format (http://www.scipy-lectures.org/advanced/scipy_sparse/csc_matrix.html) to ensure the XGBoost classifier deals well with sparse data. We don't want to just convert everything to a dense representation since then we will lose the ability to analyze features with zero value, and differentiate missing features. 

In [ ]:
class CSCTransformer:
    def transform(self, xs):
        # work around https://github.com/dmlc/xgboost/issues/1238#issuecomment-243872543
        return xs.tocsc()
    def fit(self, *args):
        return self

clf = XGBClassifier()
vec = DictVectorizer()
pipeline = make_pipeline(vec, CSCTransformer(), clf)

def evaluate(_clf):
    scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(train_xs, train_ys)  # so that parts of the original pipeline are fitted
    return np.mean(scores), _clf.predict(all_xs_test)

score_1, preds_1 = evaluate(pipeline)

### Now let's check the feature importances for these basic features

In [ ]:
show_weights(clf, vec=vec)

### The *show_predictions* method allows us to examine individual predictions

In [ ]:
show_prediction(clf, valid_xs[1], vec=vec, show_feature_values=True)

### So, if you're classified as a Female and you have paid a higher fare, you are more likely to survive than a passenger in 3rd class. We can also just examine the individual predictions but just for features without missing values. 

In [ ]:
no_missing = lambda feature_name, feature_value: not np.isnan(feature_value)
show_prediction(clf, valid_xs[1], vec=vec, show_feature_values=True, feature_filter=no_missing)

### We don't want to process name as a categorical variable

In [ ]:
vec2 = FeatureUnion([
    ('Name', CountVectorizer(
        analyzer='char_wb',
        ngram_range=(3, 4),
        preprocessor=lambda x: x['Name'],
        max_features=100,
    )),
    ('All', DictVectorizer()),
])
clf2 = XGBClassifier()

pipeline2 = make_pipeline(vec2, CSCTransformer(), clf2)

score_2, preds_2 = evaluate(pipeline2)

### Feature importances

In [ ]:
show_weights(clf2, vec=vec2)

### Some name-based features have some weightage

In [ ]:
# We hide missing features 
for idx in [4, 5, 7, 37, 81]:
    display(show_prediction(clf2, valid_xs[idx], vec=vec2,
                            show_feature_values=True, feature_filter=no_missing))

### Idea for submission filename drawn from Chia-ta Tsai's kernel https://www.kaggle.com/cttsai/forked-lgbm-w-ideas-from-kernels-and-discuss. Obviously the LB score will improve a lot if we do a little more parameter tuning + feature engineering, but I just wanted to understand how ELI5 works for now. 

In [ ]:
df_sub = pd.read_csv('../input/gender_submission.csv')

filename = 'subm_{:.6f}_{}.csv'.format(score_2, 
                     dt.now().strftime('%Y-%m-%d-%H-%M'))
print('save to {}'.format(filename))

submission = pd.DataFrame()
submission['PassengerId'] = df_sub['PassengerId']
submission['Survived'] = preds_2

submission.to_csv(filename, index=False)